In [1]:
import numpy as np
import pandas as pd
from dobbel import dobbellogger

In [2]:
dob = dobbellogger()
dob.connect()
dob.log(5, 200, 4, 125)
dob.download()
data = dob.datadf

Logging data for 5s
Done! The data is located in self.datadf


In [3]:
def expq(vector, multiplier=1):
    x, y, z = vector[0] * multiplier, vector[1] * multiplier, vector[2]  * multiplier
    absq = (x ** 2 + y ** 2 + z ** 2) ** 1/2
    w = np.cos(absq)
    sinabs = np.sin(absq)
    xq = x * sinabs / absq
    yq = y * sinabs / absq
    zq = z * sinabs / absq
    return np.array([w, xq, yq, zq])

In [4]:
def quat_mul(p, q):
    p0 = p[0]
    q0 = q[0]
    pv = p[1:]
    qv = q[1:]
    out0 = p0 * q0 - np.dot(pv, qv)
    outv = p0 * qv + q0 * pv + np.cross(pv, qv)
    out = np.array([out0, *outv])
    return out

In [5]:
def quaternion_conjugate(q):
    w, x, y, z = q
    return np.array([w, -x, -y, -z])

In [6]:
# Actual Rotation Fuction - Input = Acceleration vector = 1x3 , rotation_quaternion = 1*4
def rotate_vector(vector, rotation_quaternion):
    # Convert the vector to a quaternion
    vector_quaternion = np.concatenate(([0], vector))

    # Calculate the rotated quaternion
    rotated_quaternion = quat_mul(rotation_quaternion, quat_mul(vector_quaternion, quaternion_conjugate(rotation_quaternion)))

    # Extract the rotated vector from the quaternion
    rotated_vector = rotated_quaternion[1:]

    return np.array(rotated_vector)

In [7]:
def quat_to_euler(q):
    w, x, y, z = q[0], q[1], q[2], q[3]
    psi = np.arctan((2*x*y-2*w*z)/((2*w**2)+(2*x**2)-1))
    theta = -np.arcsin(2*x*z+2*w*y)
    phi = np.arctan((2*y*z-2*w*x)/((2*w**2)+(2*z**2)-1))
    return np.array([psi, theta, phi])

In [8]:
def remove_nan(df):
    counter = 0
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter += 1
    for i in range(counter - 1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    counter = len(df) - 1
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter -= 1
    maxlen = len(df)
    for i in range(maxlen - 1, counter + 1, -1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        df['timestamp'][i] = df['timestamp'][i] - df['timestamp'][0]
    for i in range(len(df)):
        for column in df.columns:
            if np.isnan(df[column][i]):
                deler = (df['timestamp'][i] - df['timestamp'][i - 1]) / (df['timestamp'][i + 1] - df['timestamp'][i - 1])
                df[column][i] = df[column][i - 1] + deler * (df[column][i + 1] - df[column][i - 1])
    return df

In [9]:
def left_quat_mul(q):
    q0, q1, q2, q3 = q
    lqm = np.array([
        [q0, -q1, -q2, -q3],
        [q1, 1, -q3, -q2],
        [q2, q3, 1, -q1],
        [q3, -q2, q1, 1]
    ])
    return lqm

def right_quat_mul(q):
    q0, q1, q2, q3 = q
    lqm = np.array([
        [q0, -q1, -q2, -q3],
        [q1, 1, q3, q2],
        [q2, -q3, 1, q1],
        [q3, q2, -q1, 1]
    ])
    return lqm

In [10]:
angle = np.deg2rad(-39)
rotate_y = np.array([[np.cos(angle),0,np.sin(angle)], [0,1,0],[-np.sin(angle),0,np.cos(angle)]])
trace = np.trace(rotate_y)
q_0 = np.sqrt(1+trace)/2
q_2 = 1/(4*q_0) * (np.sin(angle)+np.sin(angle))
q_rotate = np.array([q_0,0,q_2,0])

In [11]:
data = remove_nan(dob.datadf)

for i, row in data.iterrows():
    data.loc[i, 'x_acc'], data.loc[i, 'y_acc'], data.loc[i, 'z_acc'] = rotate_vector(np.array([row['x_acc'], row['y_acc'], row['z_acc']]), q_rotate)
    data.loc[i, 'x_gyro'], data.loc[i, 'y_gyro'], data.loc[i, 'z_gyro'] = rotate_vector(np.array([row['x_gyro'], row['y_gyro'], row['z_gyro']]), q_rotate)

y11 = np.array([data['x_acc'][0], data['y_acc'][0], data['z_acc'][0]])
gb = y11 / np.linalg.norm(y11)
gn = np.array([0, 0, 1])
mn = np.array([1, 0, 0])
mb = np.array([1, 0, 0])
A = - np.matmul(left_quat_mul(np.array([0, *gn])), right_quat_mul(np.array([0, *gb]))) - np.matmul(left_quat_mul(np.array([0, *mn])), right_quat_mul(np.array([0, *mb])))



# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(A)

# Eigenvalues are sorted in ascending order by np.linalg.eigh, 
# and the corresponding eigenvectors are columns of 'eigenvectors'
max_eigenvalue_index = np.argmax(eigenvalues)
max_eigenvector = eigenvectors[:, max_eigenvalue_index]

print(max_eigenvector)

[ 0.83848056 -0.53001326 -0.07088346 -0.10493728]


In [12]:
qtmin1tmin1 = max_eigenvector

dqde = np.array([
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1]
])

sigma_etha = np.array([
    [np.pi / 9, 0, 0],
    [0, np.pi / 9, 0],
    [0, 0, np.pi / 9]
])

Ptmin1tmin1 = 0.25 * np.matmul(np.matmul(np.matmul(np.matmul(right_quat_mul(qtmin1tmin1), dqde), sigma_etha), dqde.T), right_quat_mul(qtmin1tmin1))

In [39]:
gyro_bias = np.array([-0.145345332, 0.46132693, 0.01679937])
acc_bias = np.array([0.00682937, 0.0169035, -0.05057431])

qtmin1tmin1 = np.reshape(qtmin1tmin1, 4)

v = np.array([0, 0, 0])
p = np.array([0, 0, 0])
q_list = [qtmin1tmin1]
p_list = [p]
v_list = [v]

# std van gyroscope
Q = np.array([
    [-0.14534332, 0, 0],
    [0, 0.46132693, 0],
    [0, 0, 0.01679937]
])

# std van accelerometer
R = np.array([
    [-0.00014147, 0, 0],
    [0, 0.00180437, 0],
    [0, 0, 0.00452501],
])

for i in range(1, len(data['timestamp'])):
    dt = (data['timestamp'][i] - data['timestamp'][i - 1]) / 1000
    gyro_measurement = np.array([data['x_gyro'][i], data['y_gyro'][i], data['z_gyro'][i]])
    omega = gyro_measurement - gyro_bias
    expq_omega = expq(omega, 0.5 * dt * np.pi / 180)
    
    Ftmin1 = right_quat_mul(expq_omega)
    Gtmin1 = - 0.5 * dt * np.matmul(left_quat_mul(qtmin1tmin1), dqde)
    
    Pttmin1 = np.matmul(np.matmul(Ftmin1, Ptmin1tmin1), Ftmin1.T) + np.matmul(np.matmul(Gtmin1, Q), Gtmin1.T)
    
    qttmin1 = quat_mul(qtmin1tmin1, expq_omega)
    
    q0, q1, q2, q3 = quaternion_conjugate(qttmin1)
    # Zou kunnen dat hier de geconjugeerde genomen moet worden!!
    H = np.array([
        [2 * q2, 2 * q3, 2 * q0, 2 * q1],
        [-2 * q1, -2 * q0, 2 * q3, 2 * q2],
        [4 * q0, 0, 0, 4 * q3]
    ])
    S = np.matmul(np.matmul(H, Pttmin1), H.T) + R
    K = np.matmul(np.matmul(Pttmin1, H.T), np.linalg.inv(S))
    
    yat = np.array([data['x_acc'][i], data['y_acc'][i], data['z_acc'][i]]) - acc_bias
    yattmin1 = - rotate_vector(gn, quaternion_conjugate(qttmin1)) # Zou kunnen dat hier een min achter moet!!! en geconjugeerde
    epsilon = yat - yattmin1
    print(data['timestamp'][i], epsilon)

    q_tt_measure = np.reshape(qttmin1, (4, 1)) + np.matmul(K, np.reshape(epsilon, (3, 1)))
    P_tt_measure = Pttmin1 - np.matmul(np.matmul(K, S), K.T)
    
    qtt = q_tt_measure / ((np.linalg.norm(q_tt_measure)))
    #J = np.matmul(q_tt_measure, q_tt_measure.T) / (np.linalg.norm(q_tt_measure) ** 3)
    Ptt = P_tt_measure #np.matmul(np.matmul(J, P_tt_measure), J.T)
    
    Pttmin1 = Ptt
    qtmin1tmin1 = np.reshape(qtt, 4)
    
    q_list.append(qtmin1tmin1)
    

25.0 [ 1.19320516  0.61112893 -0.73908689]
29.0 [ 1.12124163 -0.28212913 -0.95723391]
35.0 [ 0.70157737  0.40313776 -0.85438418]
40.0 [ 0.33044084 -0.15809451 -0.74575374]
45.0 [ 1.76063285  0.39887602 -0.45831761]
50.0 [ 0.84718751  0.85244122 -0.44321524]
54.0 [0.27955271 0.67054124 0.08419999]
60.0 [0.0085181  0.16434551 0.11535252]
65.0 [-0.01448092  0.04171363  0.03640362]
70.0 [-0.01771041 -0.00213997  0.01505073]
75.0 [-0.01611681 -0.0087357   0.01575881]
81.0 [-0.01468581 -0.00122387  0.01057663]
85.0 [-0.01343374 -0.00834502  0.01190965]
89.0 [-0.01452559 -0.00184616  0.01608481]
91.0 [-0.01403991 -0.00345367  0.0123747 ]
95.0 [-0.01292483 -0.00454521  0.01042227]
100.0 [-0.01101048 -0.0064769   0.01515366]
106.0 [-0.01385127  0.00259513  0.01164265]
110.0 [-0.01296697 -0.00996554  0.00879935]
116.0 [-0.01158302 -0.00394999  0.01030084]
120.0 [-0.01332366 -0.00167576  0.01079599]
126.0 [-0.0163843  -0.00726548  0.01341066]
130.0 [-0.01307139 -0.00812204  0.01542167]
135.0 [-0.

In [40]:
for q, timestamp in zip(q_list, data['timestamp']):
    print(timestamp, quat_to_euler(q) * 180 / np.pi)

0.0 [-73.17240283  36.94345115  22.60044443]
25.0 [15.70946967  3.95299718 16.12793652]
29.0 [-26.52759393 -27.78119647 -15.19670772]
35.0 [-68.01908074  -9.53807811  40.87119351]
40.0 [-58.60508652  -4.63757664  62.6793039 ]
45.0 [-28.14615074 -50.79728379 -45.69257101]
50.0 [28.81636771 72.74066198 73.11139573]
54.0 [-88.19289311  62.87867954 -75.1996543 ]
60.0 [ 88.45564905  52.69921731 -86.54127893]
65.0 [ 89.32412855  49.63698539 -88.57324815]
70.0 [ 89.6072782   49.39962353 -88.600917  ]
75.0 [ 89.24465993  49.55518513 -88.68986079]
81.0 [ 89.67958421  49.35427338 -88.59544948]
85.0 [ 89.41513496  49.56640007 -88.57726442]
89.0 [ 89.32182015  49.3190179  -88.84142478]
91.0 [ 89.39401773  49.2251923  -88.87630281]
95.0 [ 89.45401448  49.23479971 -88.83243575]
100.0 [ 88.79478953  49.30002273 -89.17458515]
106.0 [ 89.30914447  48.85584238 -89.23740921]
110.0 [ 89.14860522  49.21234502 -89.03529253]
116.0 [ 89.12394856  49.198975   -89.06178015]
120.0 [ 89.37818977  49.0272499  -89.